In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

In [148]:
# Load the data from the data folder
wind_forecast = pd.read_csv('../Data/1.Wind forecast profile.csv', delimiter=';')
load = pd.read_csv('../Data/1.Load profile.csv', delimiter=';')
bus = pd.read_csv('../Data/B (power transfer factor of each bus to each line).csv', delimiter=';')
max_prod = pd.read_csv('../Data/Maximum production of generating units.csv', delimiter=';')
min_prod = pd.read_csv('../Data/Minimum production of generating units.csv', delimiter=';')
min_down_time = pd.read_csv('../Data/Minimum down time of generating units.csv', delimiter=';')
min_up_time = pd.read_csv('../Data/Minimum up time of generating units.csv', delimiter=';')
prod_cost = pd.read_csv('../Data/Production cost of generating units.csv', delimiter=';')
ramp_rate = pd.read_csv('../Data/Ramping rate of generating units.csv', delimiter=';')
start_up_cost = pd.read_csv('../Data/Start-up cost of generating units.csv', delimiter=';')
transmission_cap = pd.read_csv('../Data/Transmission capacity of lines.csv', delimiter=';')


In [149]:
# Define the input data class
class InputData:
    
    def __init__(
        self,
        wind_forecast: pd.DataFrame, 
        bus: pd.DataFrame,
        load: pd.DataFrame,
        max_prod: pd.DataFrame,
        min_prod: pd.DataFrame,
        min_down_time: pd.DataFrame,
        min_up_time: pd.DataFrame,
        prod_cost: pd.DataFrame,
        ramp_rate: pd.DataFrame,
        start_up_cost: pd.DataFrame,
        transmission_cap: pd.DataFrame
    ):
        self.time = wind_forecast['Hour']  #maybe define it with lenght of wind_production
        self.wind_forecast = wind_forecast
        self.bus = bus
        self.load = load
        self.max_prod = max_prod
        self.min_prod = min_prod
        self.min_down_time = min_down_time
        self.min_up_time = min_up_time
        self.prod_cost = prod_cost
        self.ramp_rate = ramp_rate
        self.start_up_cost = start_up_cost
        self.transmission_cap = transmission_cap
        self.M = 1000000  # Penalty for having flexible demand
        
        


In [150]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [153]:
# Define the optimization model class

class EconomicDispatch():
        
        def __init__(self, input_data: InputData):
            self.data = input_data 
            self.variables = Expando()
            self.constraints = Expando() 
            self.results = Expando() 
            self._build_model() 
            
        def _build_variables(self):
            # one binary variable for the status of each generator
            self.variables.status = {
                (i, t): self.model.addVar(vtype=GRB.BINARY, 
                                            name='status_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # one variable for each generator for each time of the day
            self.variables.prod_gen = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.max_prod.iloc[i-1, 0], 
                                           name='generation_G{}_{}'.format(i, t)) 
                                           for i in range(1, len(self.data.max_prod)+1) 
                                           for t in self.data.time}
            
            # one variable for each wind generator for each time of the day
            self.variables.prod_wind = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.wind_forecast.iloc[t, i], 
                                            name='wind_generation_W{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.wind_forecast.iloc[0, :])) 
                                            for t in self.data.time}
            
            # one variable for each start-up cost for each generator
            self.variables.start_up_cost = {
                 (i, t): self.model.addVar(lb=0, 
                                            name='start_up_cost_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # add two slack variables to always make the model feasible, allowing the demand to be flexible
            self.variables.epsilon = {
                t: self.model.addVar(lb=0, name='epsilon_{}'.format(t)) 
                for t in self.data.time}
            self.variables.delta = {
                t: self.model.addVar(lb=0, name='delta_{}'.format(t))
                for t in self.data.time}
            
            
        def _build_constraints(self):
            # Minimum capacity of the generator
            self.constraints.min_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] >= self.data.min_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}
            # Maximum capacity of the generator
            self.constraints.max_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] <= self.data.max_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}

            # Power balance constraint
            self.constraints.power_balance = {
                t: self.model.addConstr(
                    gp.quicksum(self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1)) + 
                    gp.quicksum(self.variables.prod_wind[i, t] for i in range(1, len(self.data.wind_forecast.iloc[0, :]))) == 
                    gp.quicksum(self.data.load.iloc[t, i] for i in range(1, len(self.data.load.iloc[0, :]))) +
                    self.variables.epsilon[t] - self.variables.delta[t]
                ) for t in self.data.time}

            # Transmission capacity constraint


            #Start-up costs constraint
            self.constraints.start_up_cost = {
                (i, t): self.model.addConstr(
                    self.variables.start_up_cost[i, t] >= self.data.start_up_cost.iloc[i-1, 0] * (self.variables.status[i, t] - self.variables.status[i, t-1])
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.start_up_cost_0 = {
                i: self.model.addConstr(
                    self.variables.start_up_cost[i, 0] >= self.data.start_up_cost.iloc[i-1, 0] * self.variables.status[i, 0]
                ) for i in range(1, len(self.data.max_prod)+1)}
            
            # Ramping constraint
            self.constraints.ramping_up = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] - self.variables.prod_gen[i, t-1] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.ramping_down = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t-1] - self.variables.prod_gen[i, t] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            
            # Minimum up time constraint
            self.constraints.min_up_time = {
                (i, t, to): self.model.addConstr(
                    -self.variables.status[i, t - 1] + self.variables.status[i, t] - self.variables.status[i, to] <= 0
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_up_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            
            # Minimum down time constraint
            self.constraints.min_down_time = {
                (i, t, to): self.model.addConstr(
                    self.variables.status[i, t - 1] - self.variables.status[i, t] + self.variables.status[i, to] <= 1
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_down_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            
            
            


            
            
            


        def _build_objective(self):
            # Objective function
            self.model.setObjective(
                gp.quicksum(self.data.prod_cost.iloc[i-1, 0]*self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                gp.quicksum(self.variables.start_up_cost[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                self.data.M * (gp.quicksum(self.variables.epsilon[t] for t in self.data.time) + gp.quicksum(self.variables.delta[t] for t in self.data.time))
            )

        def _build_model(self):
            self.model = gp.Model('EconomicDispatch')
            self._build_variables()
            self._build_constraints()
            self._build_objective()
            self.model.update()

        def optimize(self):
            self.model.optimize()
            self._extract_results()

        def _extract_results(self):
            self.results.production = pd.DataFrame({
                'time': [t for t in self.data.time],
                'status 1': [self.variables.status[1, t].x for t in self.data.time],
                'status 2': [self.variables.status[2, t].x for t in self.data.time],
                'status 3': [self.variables.status[3, t].x for t in self.data.time],
                'start_up_cost 1': [self.variables.start_up_cost[1, t].x for t in self.data.time],
                'start_up_cost 2': [self.variables.start_up_cost[2, t].x for t in self.data.time],
                'start_up_cost 3': [self.variables.start_up_cost[3, t].x for t in self.data.time],
                'generation 1': [self.variables.prod_gen[1, t].x for t in self.data.time],
                'generation 2': [self.variables.prod_gen[2, t].x for t in self.data.time],
                'generation 3': [self.variables.prod_gen[3, t].x for t in self.data.time],
                'wind generation 1': [self.variables.prod_wind[1, t].x for t in self.data.time],
                'wind generation 2': [self.variables.prod_wind[2, t].x for t in self.data.time],
                'load 1': [self.data.load.iloc[t, 1] for t in self.data.time],
                'load 2': [self.data.load.iloc[t, 2] for t in self.data.time],
                'load 3': [self.data.load.iloc[t, 3] for t in self.data.time],
                'epsilon': [self.variables.epsilon[t].x for t in self.data.time],
                'delta': [self.variables.delta[t].x for t in self.data.time]
            })
            
                
            
                 

In [154]:
# Run the model
input_data = InputData(wind_forecast, bus, load, max_prod, min_prod, min_down_time, min_up_time, prod_cost, ramp_rate, start_up_cost, transmission_cap)
model = EconomicDispatch(input_data)
model.optimize()
model.results.production


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 273 rows, 130 columns and 634 nonzeros
Model fingerprint: 0x48ccbd57
Variable types: 100 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 2.772000e+09
Presolve removed 110 rows and 41 columns
Presolve time: 0.01s
Presolved: 163 rows, 89 columns, 450 nonzeros
Variable types: 50 continuous, 39 integer (39 binary)

Root relaxation: objective 1.250586e+08, 43 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2506e+08    

,time,status 1,status 2,status 3,start_up_cost 1,start_up_cost 2,start_up_cost 3,generation 1,generation 2,generation 3,wind generation 1,wind generation 2,load 1,load 2,load 3,epsilon,delta
0,0,1.0,1.0,1.0,900.0,550.0,170.0,220.0,100.0,60.0,15.0,5.0,0,200,200,0.0,0.0
1,1,1.0,1.0,1.0,0.0,0.0,0.0,220.0,100.0,53.0,20.0,7.0,0,200,200,0.0,0.0
2,2,1.0,1.0,1.0,0.0,0.0,0.0,220.0,100.0,59.0,13.0,8.0,0,200,200,0.0,0.0
3,3,1.0,1.0,1.0,0.0,0.0,0.0,165.0,61.0,39.0,23.0,12.0,0,100,200,0.0,0.0
4,4,1.0,1.0,1.0,0.0,0.0,0.0,110.0,11.0,19.0,0.0,0.0,0,10,20,110.0,0.0
5,5,1.0,1.0,1.0,0.0,0.0,0.0,165.0,61.0,39.0,15.0,5.0,0,100,200,0.0,15.0
6,6,1.0,1.0,1.0,0.0,0.0,0.0,154.0,100.0,19.0,20.0,7.0,0,100,200,0.0,0.0
7,7,1.0,1.0,-0.0,0.0,0.0,0.0,179.0,100.0,0.0,13.0,8.0,0,100,200,0.0,0.0
8,8,1.0,1.0,-0.0,0.0,0.0,0.0,165.0,100.0,0.0,23.0,12.0,0,100,200,0.0,0.0
9,9,1.0,1.0,-0.0,0.0,0.0,0.0,174.0,100.0,0.0,24.0,2.0,0,100,200,0.0,0.0
